In [1]:
import pandas as pd
from matplotlib import pyplot as plt
!pip install plotly
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from datetime import *
from plotly.subplots import make_subplots

Defaulting to user installation because normal site-packages is not writeable


In [67]:
total_data = pd.read_csv('covid19cases')
total_data['date'] = pd.to_datetime(total_data.date).dt.date
sb_data = total_data[(total_data.area == 'Santa Barbara') & (total_data.area_type == 'County') & (total_data.date > date(2022, 5, 25))]
sb_data = sb_data.drop(39422, axis = 0)
sb_data['seven_day_cases_rolling_avg'] = sb_data.cases.rolling(7).mean()
sb_data['seven_day_cumulative_deaths_rolling_avg'] = sb_data.cumulative_deaths.rolling(7).mean()
sb_data['seven_day_tests_rolling_avg'] = sb_data.total_tests.rolling(7).mean()
sb_data['seven_day_test_positivity_rolling_avg'] = (sb_data.positive_tests.rolling(7).mean()/sb_data.total_tests.rolling(7).mean())*100 
sb_data = sb_data.drop(39373, axis = 0)
sb_data = sb_data.drop(39374, axis = 0)
sb_data = sb_data.drop(39375, axis = 0)
sb_data = sb_data.drop(39376, axis = 0)
sb_data = sb_data.drop(39377, axis = 0)
sb_data = sb_data.drop(39378, axis = 0)
sb_data = sb_data.astype({'seven_day_cases_rolling_avg':'int64', 'seven_day_tests_rolling_avg':'int64'})
sb_data['seven_day_cumulative_deaths_rolling_avg'] = sb_data['seven_day_cumulative_deaths_rolling_avg'].round(decimals=3)
sb_data['seven_day_test_positivity_rolling_avg'] = sb_data['seven_day_test_positivity_rolling_avg'].round(decimals=1)
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(x=list(sb_data.date),
               y=list(sb_data.cases),
               name="New Cases",
               marker_color='light blue'),
               secondary_y=False
          )
fig.add_trace(
    go.Scatter(x=list(sb_data.date),
               y=list(sb_data.seven_day_cases_rolling_avg),
               name="7-Day Average Cases",
               line=dict(color='blue'),
               visible=True),
               secondary_y=False
          )
fig.add_trace(
    go.Scatter(x=list(sb_data.date),
               y=list(sb_data.cumulative_deaths),
               name="Cumulative Deaths",
               line=dict(color='red'),
               visible=False),
               secondary_y=False
          )
fig.add_trace(
    go.Bar(x=list(sb_data.date),
               y=list(sb_data.total_tests),
               name="Tests Performed",
               marker_color='gray',
               visible=False),
               secondary_y=False
          )
fig.add_trace(
    go.Scatter(x=list(sb_data.date),
               y=list(sb_data.seven_day_tests_rolling_avg),
               name="7-Day Average Tests",
               line=dict(color='darkslategray'),
               visible=False),
               secondary_y=False
          )
fig.add_trace(
    go.Scatter(x=list(sb_data.date),
               y=list(sb_data.seven_day_test_positivity_rolling_avg),
               name="7-Day Positivity Rate (%)",
               line=dict(color='plum'), 
               visible=False),
               secondary_y=True
          )
fig.update_yaxes(title_text = 'Test Positivity Rate (%)', visible=False, secondary_y=True)
fig.update_layout(
    updatemenus=[
        dict(direction="down",
            active=0,
            x=-0.1125,
            y=1,
            buttons=list([
                dict(label="New Cases",
                     method="update",
                     args=[{"visible": [True, True, False, False, False, False, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'New Cases'},
                           'yaxis2.visible': False,
                           'title' : '<b>Santa Barbara County COVID-19 New Positive Cases Throughout First-Half of Summer', 'title.x' : 0.125,
                           'showlegend':True}]),
                dict(label="Cumulative Deaths",
                     method="update",
                     args=[{"visible": [False, False, True, False, False, False, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'Cumulative Deaths', 'range': [690, 710]},
                           'yaxis2.visible': False,
                           'title' : '<b>Santa Barbara County COVID-19 Cumulative Deaths Through First-Half of Summer', 'title.x' : 0.13,
                           'showlegend':True}]),
                dict(label="Tests Performed",
                     method="update",
                     args=[{"visible": [False, False, False, True, True, False, True]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'Tests Performed'},
                           'title' : '<b>Santa Barbara County COVID-19 Tests Performed Throughout First-Half of Summer', 'title.x' : 0.125,
                           'yaxis2.visible': False,
                           'showlegend':True}]),
                dict(label="Test Positivity",
                     method="update",
                     args=[{"visible": [False, False, False, True, False, True, True]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'Tests Performed'},
                           'title' : '<b>Santa Barbara County COVID-19 Tests Performed vs. Test Positivity Rate Throughout First-Half of Summer', 'title.x' : 0.025,
                           'yaxis2.visible': True,
                           'showlegend':True}],
                    ),
            ]),
        )
    ],
    title_text='<b>Santa Barbara County COVID-19 New Positive Cases Throughout First-Half of Summer',
            title_x=0.125,
            showlegend = True,
            legend=dict(x=1.05, y=1),
            hovermode='x unified',
            hoverlabel=dict(font_size=12, bgcolor='rgb(255, 255, 255)', namelength = -1),
            xaxis_title="Date", yaxis_title="Cases"
)
fig.update_layout(titlefont=dict(size=15, color='black'))
fig.add_annotation(text="Siddharth Chattoraj / Daily Nexus",
            showarrow=False)
fig.update_annotations(xref = 'paper', yref = 'paper', x = 0, y = -0.175)
fig.show()

In [68]:
hospital_data = pd.read_csv('hospitaldata')
hospital_data['todays_date'] = pd.to_datetime(hospital_data.todays_date).dt.date
sb_hospital = hospital_data[(hospital_data.county == 'Santa Barbara') & (hospital_data.todays_date > date(2022, 5, 31))]
fig2 = go.Figure()
fig2.add_trace(
    go.Scatter(x=list(sb_hospital.todays_date),
               y=list(sb_hospital.hospitalized_covid_confirmed_patients),
               name="Positive Patients",
               line=dict(color='Sienna'),
               visible=True),
          )
fig2.add_trace(
    go.Scatter(x=list(sb_hospital.todays_date),
               y=list(sb_hospital.hospitalized_suspected_covid_patients),
               name="Suspected Patients",
               line=dict(color='Chocolate'),
               visible=False),
          )
fig2.add_trace(
    go.Scatter(x=list(sb_hospital.todays_date),
               y=list(sb_hospital.icu_covid_confirmed_patients),
               name="ICU Positive Patients",
               line=dict(color='Peru'),
               visible=False),
          )
fig2.add_trace(
    go.Scatter(x=list(sb_hospital.todays_date),
               y=list(sb_hospital.icu_suspected_covid_patients),
               name="ICU Suspected Patients",
               line=dict(color='RosyBrown'),
               visible=False),
          )
fig2.add_trace(
    go.Scatter(x=list(sb_hospital.todays_date),
               y=list(sb_hospital.icu_available_beds),
               name="ICU Available Beds",
               line=dict(color='Tan'),
               visible=False),
          )
fig2.update_layout(
    updatemenus=[
        dict(direction="down",
            active=0,
            x=-0.1125,
            y=1,
            buttons=list([
                dict(label="Positive Patients",
                     method="update",
                     args=[{"visible": [True, False, False, False, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'Positive Patients', 'range': [0,80]},
                           'title' : '<b>Santa Barbara County Hospitalized COVID-19-Positive Patients Throughout First-Half of Summer', 'title.x': 0.075,
                           'showlegend':True}]),
                dict(label="Suspected Patients",
                     method="update",
                     args=[{"visible": [False, True, False, False, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'Suspected Patients', 'range': [0,10]},
                           'title' : '<b>Santa Barbara County Hospitalized COVID-19-Suspected Patients Throughout First-Half of Summer', 'title.x': 0.075,
                           'showlegend':True}]),
                dict(label="ICU Positive Patients",
                     method="update",
                     args=[{"visible": [False, False, True, False, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'ICU Positive Patients', 'range': [0,20]},
                           'title' : '<b>Santa Barbara County ICU Hospitalized COVID-19-Positive Patients Throughout First-Half of Summer', 'title.x': 0.075,
                           'showlegend':True}]),
                dict(label="ICU Suspected Patients",
                     method="update",
                     args=[{"visible": [False, False, False, True, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'ICU Suspected Patients', 'range': [0,10]},
                           'title' : '<b>Santa Barbara County ICU Hospitalized COVID-19-Suspected Patients Throughout First-Half of Summer', 'title.x': 0.075,
                           'showlegend':True}]),
                dict(label="ICU Beds Available",
                     method="update",
                     args=[{"visible": [False, False, False, False, True]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'ICU Hospital Beds Available', 'range': [0,50]},
                           'title' : '<b>Santa Barbara County ICU Hospital Beds Available Throughout First-Half of Summer', 'title.x': 0.15,
                           'showlegend':True}],
                    ),
            ]),
        )
    ],
    title_text='<b>Santa Barbara County Hospitalized COVID-19-Positive Patients Throughout First-Half of Summer',
            title_x=0.075,        
            showlegend = True,
            hovermode='x unified',
            hoverlabel=dict(font_size=12, bgcolor='rgb(255, 255, 255)', namelength = -1),
            xaxis_title="Date", yaxis_title="Positive Patients"
)
fig2.update_yaxes(range=[0, 80])
fig2.update_layout(titlefont=dict(size=15, color='black'))
fig2.add_annotation(text="Siddharth Chattoraj / Daily Nexus",
            showarrow=False)
fig2.update_annotations(xref = 'paper', yref = 'paper', x = 0, y = -0.175)
fig2.show()

In [69]:
ucsb_data = pd.read_csv('ucsbdata')
ucsb_data['Date'] = pd.to_datetime(ucsb_data.Date).dt.date
fig3 = go.Figure()
fig3.add_trace(
    go.Scatter(x=list(ucsb_data.Date),
               y=list(ucsb_data.New_Cases),
               name="New Cases",
               line=dict(color='SteelBlue'),
               visible=True),
          )
fig3.add_trace(
    go.Scatter(x=list(ucsb_data.Date),
               y=list(ucsb_data.Tests),
               name="Tests Performed",
               line=dict(color='CadetBlue'),
               visible=False),
          )
fig3.add_trace(
    go.Scatter(x=list(ucsb_data.Date),
               y=list(ucsb_data.Self_Reported_Off_Campus_Positive_Tests),
               name="Self-Reported Cases",
               line=dict(color='LightSteelBlue'),
               visible=False),
          )
fig3.update_layout(
    updatemenus=[
        dict(direction="down",
            active=0,
            x=-0.1125,
            y=1,
            buttons=list([
                dict(label="New Cases",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'New Cases'},
                           'title' : '<b>UCSB COVID-19 New Positive Cases Throughout First-Half of Summer', 'title.x': 0.25,
                           'showlegend':True}]),
                dict(label="Tests Performed",
                     method="update",
                     args=[{"visible": [False, True, False]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'Tests Performed'},
                           'title' : '<b>UCSB COVID-19 Tests Performed Throughout First-Half of Summer', 'title.x': 0.25,
                           'showlegend':True}]),
                dict(label="Self-Reported Cases",
                     method="update",
                     args=[{"visible": [False, False, True]},
                           {'xaxis': {'title': 'Date'},
                           'yaxis': {'title': 'Self-Reported Cases'},
                           'title' : '<b>UCSB COVID-19 New Self-Reported Positive Cases Throughout First-Half of Summer', 'title.x': 0.15,
                           'showlegend':True}],
                    ),
            ]),
        )
    ],
    title_text='<b>UCSB COVID-19 New Positive Cases Throughout First-Half of Summer',
            title_x=0.25,
            showlegend = True,
            hovermode='x unified',
            hoverlabel=dict(font_size=12, bgcolor='rgb(255, 255, 255)', namelength = -1),
            xaxis_title="Date", yaxis_title="New Cases"
)
fig3.update_layout(titlefont=dict(size=15, color='black'))
fig3.add_annotation(text="Siddharth Chattoraj / Daily Nexus",
            showarrow=False)
fig3.update_annotations(xref = 'paper', yref = 'paper', x = 0, y = -0.175)
fig3.show()

In [79]:
!pip install chart_studio
import chart_studio
username = #make plotly account user
api_key = #make plotly account api key and regenerate
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
import chart_studio.plotly as py
import chart_studio.tools as tls

Defaulting to user installation because normal site-packages is not writeable


In [70]:
py.plot(fig, filename = 'Plot 1 Actual', auto_open=True)

'https://plotly.com/~yeahsidc/8/'

In [71]:
py.plot(fig2, filename = 'Plot 2 Actual', auto_open=True)

'https://plotly.com/~yeahsidc/10/'

In [72]:
py.plot(fig3, filename = 'Plot 3 Actual', auto_open=True)

'https://plotly.com/~yeahsidc/12/'

In [73]:
tls.get_embed('https://plotly.com/~yeahsidc/8/') 

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~yeahsidc/8.embed" height="525" width="100%"></iframe>'

In [74]:
tls.get_embed('https://plotly.com/~yeahsidc/10/') 

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~yeahsidc/10.embed" height="525" width="100%"></iframe>'

In [75]:
tls.get_embed('https://plotly.com/~yeahsidc/12/') 

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~yeahsidc/12.embed" height="525" width="100%"></iframe>'